In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [4]:
!pip install wandb

In [5]:
import wandb
wandb.login()

wandb: Currently logged in as: rochelleli (use `wandb login --relogin` to force relogin)


True

In [6]:
wandb.init(project="w251-hw9", entity="rochelleli")

In [7]:
SEED=1

random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [8]:
torch.cuda.device_count()

1

In [9]:
START_EPOCH = 0

In [10]:
ARCH = 'resnet18'
EPOCHS = 2
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 10
TRAIN_BATCH=500
VAL_BATCH=500
WORKERS=2
TRAINDIR="/home/ubuntu/data/train"
VALDIR="/home/ubuntu/data/val"

In [11]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [12]:
wandb.init(config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH})

In [13]:
GPU = 0

In [14]:
torch.cuda.set_device(GPU)

In [15]:
global_step = 0

In [16]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        optimizer.zero_grad()

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
          output = model(images)
          loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})
        
        global_step = global_step + 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [17]:
def validate(val_loader, model, criterion):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)    
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    
    global_step = global_step + 1

    return top1.avg

In [18]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [19]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [20]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [21]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [22]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [23]:
cudnn.benchmark = True

In [24]:
import torch.distributed as dist
import sys

In [25]:
WORLD_SIZE = 2
BACKEND = 'nccl'

URL = 'tcp://172.31.17.123:443'

RANK = 0

dist.init_process_group(backend = BACKEND, init_method=URL,rank=RANK, world_size=WORLD_SIZE)

In [26]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [27]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [28]:
IMG_SIZE = 224
NUM_CLASSES = 1000

In [29]:
model = models.__dict__[ARCH]()

In [30]:
inf = model.fc.in_features

In [31]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [32]:
model.cuda(GPU)
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

In [33]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [34]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [35]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [36]:
transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [37]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [38]:
transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [39]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [40]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [41]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [42]:
best_acc1 = 0

In [43]:
for epoch in range(START_EPOCH, 2):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()[0]))
    
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    
    wandb.log({'lr': scheduler.get_last_lr()[0]})

Epoch: [0][   0/1282]	Time 31.765 (31.765)	Data  3.818 ( 3.818)	Loss 7.0052e+00 (7.0052e+00)	Acc@1   0.00 (  0.00)	Acc@5   1.00 (  1.00)
Epoch: [0][  10/1282]	Time  0.785 ( 3.937)	Data  0.096 ( 0.725)	Loss 6.9195e+00 (6.9718e+00)	Acc@1   0.60 (  0.18)	Acc@5   1.20 (  0.78)
Epoch: [0][  20/1282]	Time  0.754 ( 2.881)	Data  0.097 ( 0.677)	Loss 6.9420e+00 (6.9584e+00)	Acc@1   0.20 (  0.18)	Acc@5   0.80 (  0.81)
Epoch: [0][  30/1282]	Time  0.763 ( 2.512)	Data  0.104 ( 0.573)	Loss 6.8143e+00 (6.9239e+00)	Acc@1   0.80 (  0.26)	Acc@5   2.00 (  1.10)
Epoch: [0][  40/1282]	Time  0.758 ( 2.296)	Data  0.096 ( 0.445)	Loss 6.7460e+00 (6.8903e+00)	Acc@1   0.40 (  0.37)	Acc@5   3.20 (  1.39)
Epoch: [0][  50/1282]	Time  0.666 ( 2.180)	Data  0.002 ( 0.380)	Loss 6.7190e+00 (6.8485e+00)	Acc@1   0.20 (  0.42)	Acc@5   2.00 (  1.64)
Epoch: [0][  60/1282]	Time  0.695 ( 2.114)	Data  0.002 ( 0.321)	Loss 6.5468e+00 (6.8072e+00)	Acc@1   1.20 (  0.52)	Acc@5   4.00 (  1.86)
Epoch: [0][  70/1282]	Time  0.755 ( 2.048

Epoch: [0][ 600/1282]	Time  0.680 ( 1.730)	Data  0.002 ( 0.037)	Loss 4.6452e+00 (5.4635e+00)	Acc@1  10.80 (  6.25)	Acc@5  29.80 ( 16.98)
Epoch: [0][ 610/1282]	Time  0.680 ( 1.730)	Data  0.003 ( 0.036)	Loss 4.5628e+00 (5.4475e+00)	Acc@1  11.40 (  6.37)	Acc@5  33.00 ( 17.24)
Epoch: [0][ 620/1282]	Time  0.780 ( 1.729)	Data  0.099 ( 0.036)	Loss 4.4533e+00 (5.4322e+00)	Acc@1  14.80 (  6.49)	Acc@5  36.00 ( 17.49)
Epoch: [0][ 630/1282]	Time  0.695 ( 1.729)	Data  0.002 ( 0.037)	Loss 4.4817e+00 (5.4168e+00)	Acc@1  15.40 (  6.62)	Acc@5  34.60 ( 17.74)
Epoch: [0][ 640/1282]	Time  0.686 ( 1.727)	Data  0.002 ( 0.036)	Loss 4.5289e+00 (5.4026e+00)	Acc@1  13.40 (  6.73)	Acc@5  33.80 ( 17.98)
Epoch: [0][ 650/1282]	Time  0.684 ( 1.727)	Data  0.002 ( 0.036)	Loss 4.4358e+00 (5.3879e+00)	Acc@1  14.40 (  6.84)	Acc@5  34.20 ( 18.22)
Epoch: [0][ 660/1282]	Time  0.685 ( 1.726)	Data  0.002 ( 0.035)	Loss 4.5140e+00 (5.3730e+00)	Acc@1  11.00 (  6.96)	Acc@5  32.20 ( 18.46)
Epoch: [0][ 670/1282]	Time  0.782 ( 1.725

Epoch: [0][1200/1282]	Time  0.780 ( 1.704)	Data  0.097 ( 0.041)	Loss 3.4662e+00 (4.7511e+00)	Acc@1  30.20 ( 12.77)	Acc@5  50.60 ( 29.06)
Epoch: [0][1210/1282]	Time  0.687 ( 1.704)	Data  0.003 ( 0.042)	Loss 3.6185e+00 (4.7416e+00)	Acc@1  25.80 ( 12.87)	Acc@5  48.00 ( 29.23)
Epoch: [0][1220/1282]	Time  0.689 ( 1.704)	Data  0.002 ( 0.042)	Loss 3.5822e+00 (4.7323e+00)	Acc@1  25.80 ( 12.98)	Acc@5  48.80 ( 29.41)
Epoch: [0][1230/1282]	Time  0.716 ( 1.704)	Data  0.003 ( 0.042)	Loss 3.5782e+00 (4.7228e+00)	Acc@1  25.40 ( 13.08)	Acc@5  50.20 ( 29.57)
Epoch: [0][1240/1282]	Time  0.714 ( 1.704)	Data  0.003 ( 0.041)	Loss 3.7009e+00 (4.7135e+00)	Acc@1  22.40 ( 13.18)	Acc@5  48.20 ( 29.74)
Epoch: [0][1250/1282]	Time  0.724 ( 1.703)	Data  0.003 ( 0.041)	Loss 3.6202e+00 (4.7043e+00)	Acc@1  26.60 ( 13.28)	Acc@5  49.60 ( 29.90)
Epoch: [0][1260/1282]	Time  0.721 ( 1.704)	Data  0.003 ( 0.041)	Loss 3.4213e+00 (4.6952e+00)	Acc@1  28.80 ( 13.37)	Acc@5  54.00 ( 30.06)
Epoch: [0][1270/1282]	Time  0.785 ( 1.703

Epoch: [1][ 430/1282]	Time  2.582 ( 1.669)	Data  1.864 ( 0.974)	Loss 3.1829e+00 (3.2181e+00)	Acc@1  31.00 ( 31.52)	Acc@5  59.20 ( 57.05)
Epoch: [1][ 440/1282]	Time  2.590 ( 1.670)	Data  1.856 ( 0.974)	Loss 3.1508e+00 (3.2153e+00)	Acc@1  32.20 ( 31.56)	Acc@5  58.20 ( 57.11)
Epoch: [1][ 450/1282]	Time  2.437 ( 1.670)	Data  1.759 ( 0.974)	Loss 3.1292e+00 (3.2129e+00)	Acc@1  34.20 ( 31.62)	Acc@5  58.20 ( 57.14)
Epoch: [1][ 460/1282]	Time  2.658 ( 1.671)	Data  1.908 ( 0.975)	Loss 3.1293e+00 (3.2096e+00)	Acc@1  33.20 ( 31.66)	Acc@5  58.40 ( 57.22)
Epoch: [1][ 470/1282]	Time  2.677 ( 1.671)	Data  1.924 ( 0.975)	Loss 2.9390e+00 (3.2069e+00)	Acc@1  34.20 ( 31.69)	Acc@5  61.80 ( 57.26)
Epoch: [1][ 480/1282]	Time  2.769 ( 1.672)	Data  2.056 ( 0.976)	Loss 3.0683e+00 (3.2039e+00)	Acc@1  35.80 ( 31.74)	Acc@5  59.60 ( 57.32)
Epoch: [1][ 490/1282]	Time  2.575 ( 1.672)	Data  1.847 ( 0.976)	Loss 2.9745e+00 (3.2007e+00)	Acc@1  37.40 ( 31.81)	Acc@5  61.20 ( 57.36)
Epoch: [1][ 500/1282]	Time  2.954 ( 1.673

Epoch: [1][1030/1282]	Time  2.512 ( 1.673)	Data  1.796 ( 0.973)	Loss 2.8795e+00 (3.0650e+00)	Acc@1  36.20 ( 33.93)	Acc@5  65.40 ( 59.78)
Epoch: [1][1040/1282]	Time  2.767 ( 1.673)	Data  2.047 ( 0.973)	Loss 2.9928e+00 (3.0631e+00)	Acc@1  34.40 ( 33.96)	Acc@5  61.20 ( 59.82)
Epoch: [1][1050/1282]	Time  2.525 ( 1.673)	Data  1.801 ( 0.972)	Loss 2.9927e+00 (3.0612e+00)	Acc@1  38.60 ( 34.00)	Acc@5  62.20 ( 59.85)
Epoch: [1][1060/1282]	Time  2.502 ( 1.673)	Data  1.799 ( 0.972)	Loss 2.7093e+00 (3.0588e+00)	Acc@1  37.60 ( 34.03)	Acc@5  66.40 ( 59.90)
Epoch: [1][1070/1282]	Time  2.814 ( 1.673)	Data  2.080 ( 0.972)	Loss 2.7651e+00 (3.0566e+00)	Acc@1  35.00 ( 34.06)	Acc@5  66.20 ( 59.94)
Epoch: [1][1080/1282]	Time  2.693 ( 1.672)	Data  1.942 ( 0.972)	Loss 2.7423e+00 (3.0545e+00)	Acc@1  40.40 ( 34.10)	Acc@5  66.40 ( 59.97)
Epoch: [1][1090/1282]	Time  2.513 ( 1.672)	Data  1.838 ( 0.972)	Loss 2.7287e+00 (3.0525e+00)	Acc@1  42.00 ( 34.13)	Acc@5  64.80 ( 60.01)
Epoch: [1][1100/1282]	Time  2.546 ( 1.672

In [44]:
writer.close()
%load_ext tensorboard
%tensorboard --logdir=runs